In [15]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, Var, PositiveReals, NonNegativeReals
from math import log
import pandas as pd
from pyomo.environ import RangeSet
from functools import partial
from pyomo.environ import Objective, maximize


In [16]:
class ChemicalModel:
    
    def __init__(self):
        self.model = ConcreteModel()
        self.components = ['Hydrogen', 'Methane', 'Benzene', 'Toluene', 'ParaXylene', 'Diphenyl']
        self.parameters = Parameters()
        
        # Set params as an attribute of model
        self.model.params = self.parameters.params
        
        self.variables = Variables(self.model, self.components, self.model.params)
        self.constraints = Constraints(self.model, self.model.params)
        
        # Set up the objective function
        self.set_objective()
        
    def set_objective(self):
        """Define the objective function for the model."""
        self.model.objective = Objective(expr=self.model.s13['Hydrogen'], sense=maximize)
        self.model.objective = Objective(expr=self.model.s15['Benzene'], sense=maximize)
    
    def solve(self):
        solver = SolverFactory('ipopt')
        solver.options['constr_viol_tol'] = 1e-8
        solver.options['acceptable_constr_viol_tol'] = 1e-8

        solver.solve(self.model, tee=True)

    def fetch_value(self, var):
        """Fetch the value of a variable and round it."""
        val = var.value
        if val is None or val < 0:
            return 0.0
        return round(val, 4)  

    
    def generate_stream_data(self, stream_name):
        """Generate molar flow rates for a given stream."""
        stream_index = int(stream_name[1:])  # Extract the integer value from the stream name

        if stream_name in ['s8', 's9']:
            s_flow = self.model.params[stream_name.upper()]
            molar_flow_rates = [s_flow * self.model.params[f'{stream_name.upper()}_{component}'] for component in self.components]
        else:
            molar_flow_rates = [self.fetch_value(getattr(self.model, f's{stream_index}')[component]) for component in self.components]

        return molar_flow_rates

    def generate_stream_table(self):
        """Generate a table with molar flow rates of each component in each stream."""
        data = []  # This will store rows of data which will be used to create DataFrame

        # For each stream
        for i in range(8, 19):  # Adjusted the range to start from S8
            stream_name = f's{i}'
            data.append(self.generate_stream_data(stream_name))

        # Creating DataFrame
        stream_names = [f's{i}' for i in range(8, 19)]  # Adjusted the range to start from S8
        df = pd.DataFrame(data, columns=self.components, index=stream_names)
        return df

    def display_results(self):
        # Helper function to fetch the value
        def fetch_value(var):
            val = var.value
            if val is None or val < 0:
                return 0.0
            return val

        # Helper function to display and write a set of results
        def display_and_write(file, header, results):
            file.write(header + '\n')
            print(header)
            for result in results:
                file.write(result + '\n')
                print(result)

        # Specify the filename where you want to store the results
        filename = "original_model_results.txt"
        model = self.model 

        with open(filename, 'w') as file:
            # Write the header to the file
            file.write("Results:\n")
            print("Results:")

            # Stream S Results
            s_results = [
                f"S{i}: {fetch_value(getattr(model, f'S{i}'))}" for i in range(10, 19)
            ]
            # Add results for S8 and S9 from parameters
            s_results.insert(0, f"S8: {model.params['S8']}")
            s_results.insert(1, f"S9: {model.params['S9']}")
            display_and_write(file, "\nStream S Results:", s_results)

            # Component molar flow rate results for Streams S10 to S18
            components = ['Hydrogen', 'Methane', 'Benzene', 'Toluene', 'ParaXylene', 'Diphenyl']
            molar_flowRate_results = [
                f"Molar Flow rate of[{component}] in S{i}: {fetch_value(getattr(model, f's{i}')[component])}" 
                for i in range(10, 19) for component in components
            ]
            # Add composition results for S8 and S9 from parameters
            for component in components:
                molar_flowRate_results.insert(0, f"Molar Flow rate of[{component}] in S8: {model.params['S8'] * model.params[f'S8_{component}']}")
                molar_flowRate_results.insert(1, f"Molar Flow rate of[{component}] in S9: {model.params['S9'] * model.params[f'S9_{component}']}")
            display_and_write(file, "\nComponent Flow Rate Results:", molar_flowRate_results)

            # Generate the stream table
            stream_table_df = self.generate_stream_table()
            stream_table_str = stream_table_df.to_string()

            # Print the stream table to the terminal
            print("\nStream Table:")
            print(stream_table_str)

            # Write the stream table to the file
            file.write("\nStream Table:\n")
            file.write(stream_table_str)
            file.write("\n")

        # Print to console that results are written to the file
        print(f"\nResults have been written to {filename}")


In [17]:
class Parameters:
    def __init__(self):
        self.params = {
            
            # Molar flow rate of Stream 8 (Hydrogen Feed inlet)
            'S8': 12714.60, # to the power of 10**6 [kmol / hr]
            
            # Molar flow rate of Stream 9 (Toluene Feed inlet)
            'S9': 2542.92, # to the power of 10**6 [kmol / hr]
            
            # Molar Composition of Stream 8 (Pure Hydrogen)
            'S8_Hydrogen': 1,
            'S8_Methane': 0.0,
            'S8_Benzene': 0.0,
            'S8_Toluene': 0.0 ,
            'S8_ParaXylene': 0.0,
            'S8_Diphenyl': 0.0,            
            
            # Molar composition of Stream 9 (Combined Feed inlet of Stream 4 and 6)
            'S9_Hydrogen': 0.0,
            'S9_Methane': 0.0,
            'S9_Benzene': 0.90/100,
            'S9_Toluene': 0.991 ,
            'S9_ParaXylene': 0.0,
            'S9_Diphenyl': 0.0,
            
            # Fractional recoveries of HK/LK
            'FR_S11_LK': 0.98,
            'FR_S12_HK': 0.95,
            'FR_S15_LK': 0.98,
            'FR_S16_HK': 0.95,
            'FR_S17_LK': 0.98,
            'FR_S18_HK': 0.95,
            
            # Purge Composition (Hydrogen + Benzene)
            'yPH': 0.2,
            'yPB': 0.0001,
            
            # Conversion: 
            'X': 0.5
            
        }

In [18]:
class Variables:
    def __init__(self, model, components, parameters):
        self.define_variables(model, components, parameters)

    def define_variables(self, model, components, parameters):
        initial_value = 0.00001

        # Define a set for the streams
        model.streams = RangeSet(10, 18)

        for i in model.streams:
            # Overall Stream molar flow rates
            setattr(model, f'S{i}', Var(within=NonNegativeReals, initialize=1000))

            # Individual component stream flow rates
            setattr(model, f's{i}', Var(components, within=NonNegativeReals, initialize=initial_value))

        # Individual component molar composition for all streams
        model.x = Var(model.streams, components, within=NonNegativeReals, bounds=[0, 1])
   
        # Extents of Reaction
        model.zeta_1 = Var(within=NonNegativeReals, doc='Extent of reaction 1')
        model.zeta_2 = Var(within=NonNegativeReals, doc='Extent of reaction 2')
        
        # Selectivity of Benzene
        model.S = Var(within=NonNegativeReals, doc='Selectivity of Benzene')
        
        # Conversion of Toluene
        model.X = Var(within=NonNegativeReals, bounds=[0, 1], initialize=0.9)

In [19]:
class Constraints:
    components = ['Hydrogen', 'Methane', 'Benzene', 'Toluene', 'ParaXylene', 'Diphenyl']

    def __init__(self, model, parameters=None):
        self.model = model
        if parameters:
            self.define_constraints(model, parameters)

    def define_constraints(self, model, parameters):
        
        # Check and delete existing components before redefining
        if hasattr(model, 'streams'):
            model.del_component(model.streams)
        model.streams = RangeSet(10, 18)
        
        # Physical condition (composition within a stream should add up to 1)
        model.composition_sum_constraint = Constraint(model.streams, rule=self.composition_sum_rule)
        
        
        # Add overall material balance constraints for streams S10 to S18
        for i in model.streams:  # For streams S10 to S18
            setattr(model, f'Eq0_S{i}', Constraint(expr=self.overall_material_balance(model, i)))
            
        # Map each constraint to its corresponding function
        constraints_mapping = {
            'Eq1': self.Eq1,
            'Eq2': self.Eq2,
            'Eq3': self.Eq3,
            'Eq4': self.Eq4,
            'Eq5': self.Eq5,
            'Eq6': self.Eq6,
            'Eq7': self.Eq7,
            'Eq8': self.Eq8,
            'Eq9': self.Eq9,
            'Eq10': self.Eq10,
            'Eq11': self.Eq11,
            'Eq12': self.Eq12,
            'selectivity_def_constraint': self.selectivity_def_rule
        }

        for component in self.components:
            # Dynamically create constraint name based on the component
            for i in range(1, 11):
                rule_name = f'{component}_comp_rule{i}'
                if hasattr(self, rule_name):
                    constraints_mapping[rule_name] = getattr(self, rule_name)

        # Dynamically add each constraint to the model
        for constraint_name, rule_function in constraints_mapping.items():
            setattr(model, constraint_name, Constraint(rule=rule_function))
                
                
    
    # Overall Material Balance
    def overall_material_balance(self, model, stream):
        return getattr(model, f'S{stream}') == sum(getattr(model, f's{stream}')[component] for component in self.components)

    # Physical condition 
    def component_molar_composition(self, model, stream, component):
        return model.x[stream, component] == getattr(model, f's{stream}')[component] / getattr(model, f'S{stream}')

    # Physical condition (composition within a stream should add up to 1)
    def composition_sum_rule(self, model, stream):
        return sum(model.x[stream, component] for component in self.components) == 1.0

    # Fractional Recovery (HK/LK) Equations   
    def Eq1(self, model):
        return model.params['FR_S11_LK'] * model.s10['Methane'] == model.s11['Methane'] 
    
    def Eq2(self, model):
        return (1 - model.params['FR_S11_LK']) * model.s10['Methane'] == model.s12['Methane']
    
    def Eq3(self, model):
        return model.params['FR_S12_HK'] * model.s10['Benzene'] == model.s12['Benzene'] 
    
    def Eq4(self, model):
        return (1 - model.params['FR_S12_HK']) * model.s10['Benzene'] == model.s11['Benzene'] 
    
    def Eq5(self, model):
        return model.params['FR_S15_LK'] * model.s12['Benzene'] == model.s15['Benzene']
    
    def Eq6(self, model):
        return (1 - model.params['FR_S15_LK']) * model.s12['Benzene']  == model.s16['Benzene'] 
    
    def Eq7(self, model):
        return model.params['FR_S16_HK'] * model.s12['Toluene'] == model.s16['Toluene']
    
    def Eq8(self, model):
        return (1 - model.params['FR_S16_HK']) * model.s12['Toluene'] == model.s15['Toluene'] 
    
    def Eq9(self, model):
        return model.params['FR_S17_LK'] * model.s16['Toluene']  == model.s17['Toluene'] 
    
    def Eq10(self, model):
        return (1 - model.params['FR_S17_LK']) * model.s16['Toluene']  == model.s18['Toluene'] 
    
    def Eq11(self, model):
        return model.params['FR_S18_HK'] * model.s16['ParaXylene']  == model.s18['ParaXylene'] 
    
    def Eq12(self, model):
        return (1 - model.params['FR_S18_HK']) * model.s16['ParaXylene']  == model.s17['ParaXylene'] 
    
    
    # Selectivity Definition
    def selectivity_def_rule(self, model):
        return model.S * (1 - model.params['X'])**(1.544) == (1 - model.params['X'])**(1.544) - 0.0036 
    
    def consumption_of_Toluene(self, model):
        return C + model.s10['Toluene'] == model.params['S9'] * model.params['S9_Toluene'] + model.params['S8'] * model.params['S8_Toluene'] + model.s13['Toluene'] + model.s17['Toluene']
    
    # Selectivity relation
    def selectivity_relation(self, model):
        return model.C * model.S == model.s10['Benzene']
    
    # Hydrogen (Individual Component material balance)
    def Hydrogen_comp_rule1(self, model):
        return model.params['S9'] * model.params['S9_Hydrogen'] + model.params['S8'] * model.params['S8_Hydrogen'] + model.s13['Hydrogen'] + model.s17['Hydrogen'] - model.zeta_1 + model.zeta_2 == model.s10['Hydrogen'] 

    def Hydrogen_comp_rule2(self, model):
        return model.s10['Hydrogen']  ==  model.s11['Hydrogen']
    
    def Hydrogen_comp_rule3(self, model):
        return model.s11['Hydrogen'] ==  model.s13['Hydrogen'] + model.s14['Hydrogen']
        
    def Hydrogen_comp_rule4(self, model):
        return model.s14['Hydrogen'] == model.params['yPH'] * model.S14
    
    def Hydrogen_comp_rule5(self, model):
        return model.s12['Hydrogen'] == 0
    
    def Hydrogen_comp_rule6(self, model):
        return model.s15['Hydrogen'] == 0
    
    def Hydrogen_comp_rule7(self, model):
        return model.s16['Hydrogen'] == 0
    
    def Hydrogen_comp_rule8(self, model):
        return model.s17['Hydrogen'] == 0 
    
    def Hydrogen_comp_rule9(self, model):
        return model.s18['Hydrogen'] == 0 
    
    def Hydrogen_comp_rule10(self, model):
        return model.s13['Hydrogen'] == 5 * model.s14['Hydrogen']    
    
    # Methane (Individual Component material balance)
    def Methane_comp_rule1(self, model):
        return model.params['S9']*model.params['S9_Methane'] + model.params['S8']*model.params['S8_Methane'] + model.s13['Methane'] + model.s17['Methane'] + model.zeta_1 == model.s10['Methane']
    
    def Methane_comp_rule2(self, model):
        return model.s10['Methane'] ==  model.s11['Methane'] + model.s12['Methane']
   
    def Methane_comp_rule3(self, model):
        return model.s11['Methane'] ==  model.s13['Methane'] + model.s14['Methane']
    
    def Methane_comp_rule4(self, model):
        return model.s14['Methane'] == (1 - model.params['yPH'] - model.params['yPB']) * model.S14
       
    def Methane_comp_rule5(self, model):
        return model.s12['Methane'] == model.s15['Methane']
    
    def Methane_comp_rule6(self, model):
        return model.s16['Methane'] == 0
    
    def Methane_comp_rule7(self, model):
        return model.s17['Methane'] == 0
    
    def Methane_comp_rule8(self, model):
        return model.s18['Methane'] == 0  
    
    def Methane_comp_rule9(self, model):
        return model.s13['Methane'] == 5 * model.s14['Methane']   
    
    
    
    # Benzene (Individual Component material balance)
    def Benzene_comp_rule1(self, model):
        return model.params['S9']*model.params['S9_Benzene'] + model.params['S8']*model.params['S8_Benzene'] + model.s13['Benzene'] + model.s17['Benzene'] + model.zeta_1 - 2 * model.zeta_2 == model.s10['Benzene']
    
    def Benzene_comp_rule2(self, model):
        return model.s10['Benzene'] ==  model.s11['Benzene'] + model.s12['Benzene']
   
    def Benzene_comp_rule3(self, model):
        return model.s11['Benzene'] ==  model.s13['Benzene'] + model.s14['Benzene'] 
    
    def Benzene_comp_rule4(self, model):
        return model.s14['Benzene'] == model.params['yPB'] * model.S14
    
    def Benzene_comp_rule5(self, model):
        return model.s12['Benzene'] == model.s15['Benzene'] + model.s16['Benzene']
    
    def Benzene_comp_rule6(self, model):
        return model.s16['Benzene'] == model.s17['Benzene']
    
    def Benzene_comp_rule7(self, model):
        return model.s18['Methane'] == 0  
    
    def Benzene_comp_rule8(self, model):
        return model.s13['Benzene'] == 5 * model.s14['Benzene']
    
    
    
    # Toluene (Individual Component material balance)
    def Toluene_comp_rule1(self, model):
        return model.params['S9']*model.params['S9_Toluene'] + model.params['S8']*model.params['S8_Toluene'] + model.s13['Toluene'] + model.s17['Toluene'] - model.zeta_1 == model.s10['Toluene']
    
    
    def Toluene_comp_rule2(self, model):
        return model.s10['Toluene'] ==  model.s12['Toluene']
   
    def Toluene_comp_rule3(self, model):
        return model.s12['Toluene'] == model.s15['Toluene'] + model.s16['Toluene']
    
    def Toluene_comp_rule4(self, model):
        return model.s16['Toluene'] == model.s17['Toluene'] + model.s18['Toluene']
    
    def Toluene_comp_rule5(self, model):
        return model.s11['Toluene'] == 0   
    
    def Toluene_comp_rule6(self, model):
        return model.s13['Toluene'] == 0 
    
    def Toluene_comp_rule7(self, model):
        return model.s14['Toluene'] == 0     
    
    
    
    # ParaXylene (Individual Component material balance)
    def ParaXylene_comp_rule1(self, model):
        return model.params['S9']*model.params['S9_ParaXylene'] + model.params['S8']*model.params['S8_ParaXylene'] + model.s13['ParaXylene'] + model.s17['ParaXylene'] == model.s10['ParaXylene']
    
    def ParaXylene_comp_rule2(self, model):
        return model.s10['ParaXylene'] ==  model.s12['ParaXylene']
   
    def ParaXylene_comp_rule3(self, model):
        return model.s12['ParaXylene'] == model.s16['ParaXylene']
    
    def ParaXylene_comp_rule4(self, model):
        return model.s16['ParaXylene']  == model.s17['ParaXylene'] + model.s18['ParaXylene']
    
    def ParaXylene_comp_rule5(self, model):
        return model.s11['ParaXylene'] == 0  
    
    def ParaXylene_comp_rule6(self, model):
        return model.s13['ParaXylene'] == 0  
    
    def ParaXylene_comp_rule7(self, model):
        return model.s14['ParaXylene'] == 0  
    
    def ParaXylene_comp_rule8(self, model):
        return model.s15['ParaXylene'] == 0  
    

    # Diphenyl (Individual Component material balance)
    def Diphenyl_comp_rule1(self, model):
        return model.params['S9']*model.params['S9_Diphenyl'] + model.params['S8']*model.params['S8_Diphenyl'] + model.s13['Diphenyl'] + model.s17['Diphenyl']  + model.zeta_2 == model.s10['Diphenyl']
    
    def Diphenyl_comp_rule2(self, model):
        return model.s10['Diphenyl'] ==  model.s12['Diphenyl']
   
    def Diphenyl_comp_rule3(self, model):
        return model.s12['Diphenyl'] == model.s16['Diphenyl']
    
    def Diphenyl_comp_rule4(self, model):
        return model.s16['Diphenyl']  == model.s18['Diphenyl']
    
    def Diphenyl_comp_rule5(self, model):
        return model.s11['Diphenyl'] == 0  
    
    def Diphenyl_comp_rule6(self, model):
        return model.s13['Diphenyl'] == 0  
    
    def Diphenyl_comp_rule7(self, model):
        return model.s14['Diphenyl'] == 0  
    
    def Diphenyl_comp_rule8(self, model):
        return model.s15['Diphenyl'] == 0  
    
    def Diphenyl_comp_rule9(self, model):
        return model.s17['Diphenyl'] == 0 

In [20]:
chemical_model1 = ChemicalModel()

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [21]:
chemical_model1.solve()

Ipopt 3.14.12: constr_viol_tol=1e-08
acceptable_constr_viol_tol=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:      245
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      120
                     variables with only lower bounds:       66
                variables with lower and upper bounds:       54
                     variables with only upper bounds:        0
Total number of equal

In [22]:
chemical_model1.display_results()

Results:

Stream S Results:
S8: 12714.6
S9: 2542.92
S10: 81700.37487417593
S11: 70047.34319222353
S12: 11653.031681952472
S13: 58395.27318362677
S14: 11674.557198703878
S15: 11442.617641554709
S16: 210.41404039776725
S17: 210.4140404079288
S18: 11.926686781342045

Component Flow Rate Results:
Molar Flow rate of[Diphenyl] in S8: 0.0
Molar Flow rate of[Diphenyl] in S9: 0.0
Molar Flow rate of[ParaXylene] in S8: 0.0
Molar Flow rate of[ParaXylene] in S9: 0.0
Molar Flow rate of[Toluene] in S8: 0.0
Molar Flow rate of[Toluene] in S9: 2520.03372
Molar Flow rate of[Benzene] in S8: 0.0
Molar Flow rate of[Benzene] in S9: 22.886280000000003
Molar Flow rate of[Methane] in S8: 0.0
Molar Flow rate of[Methane] in S9: 0.0
Molar Flow rate of[Hydrogen] in S8: 12714.6
Molar Flow rate of[Hydrogen] in S9: 0.0
Molar Flow rate of[Hydrogen] in S10: 14009.468638444681
Molar Flow rate of[Methane] in S10: 56616.48305590463
Molar Flow rate of[Benzene] in S10: 11074.423179846519
Molar Flow rate of[Toluene] in S10: 0

In [23]:
chemical_model1.generate_stream_table()

,Hydrogen,Methane,Benzene,Toluene,ParaXylene,Diphenyl
s8,12714.6000,0.0000,0.00000,0.00000,0.0,0.0
s9,0.0000,0.0000,22.88628,2520.03372,0.0,0.0
s10,14009.4686,56616.4831,11074.42320,0.00000,0.0,0.0
s11,14009.4686,55484.1534,553.72120,0.00000,0.0,0.0
s12,0.0000,1132.3297,10520.70200,0.00000,0.0,0.0
s13,11674.5572,46236.7945,461.43430,22.48720,0.0,0.0
s14,2334.9114,9247.3589,92.28690,0.00000,0.0,0.0
s15,0.0000,1132.3297,10310.28800,0.00000,0.0,0.0
s16,0.0000,0.0000,210.41400,0.00000,0.0,0.0
s17,0.0000,0.0000,210.41400,0.00000,0.0,0.0
